In [ ]:
!pip uninstall -y numpy scipy autogluon autogluon.timeseries
!pip install numpy==1.23.5 scipy==1.9.3
!pip install autogluon.timeseries
!pip install torch

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import torch

# Load your preprocessed dataset
file_path = 'automl_dataset.csv'
df = pd.read_csv(file_path)

# Ensure timestamp column is datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Set all rows to the same item_id (1) to create a single time series
df['item_id'] = 1

# Convert to TimeSeriesDataFrame
df_tsd = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column='item_id',
    timestamp_column='timestamp'
)

# Split into training and testing data
train_size = int(len(df_tsd) * 0.8)
split_timestamp = df_tsd.index.get_level_values(1)[train_size]

train_data = df_tsd[df_tsd.index.get_level_values(1) < split_timestamp]
test_data = df_tsd[df_tsd.index.get_level_values(1) >= split_timestamp]

# Setup and train the TimeSeriesPredictor with reduced complexity
predictor = TimeSeriesPredictor(
    path='autogluon_checkpoints',
    prediction_length=1,
    
    eval_metric='RMSE',
    freq='D',
)

# Fit the model with reduced time limit and simpler preset
predictor.fit(
    train_data=train_data, 
    presets='high_quality',
    time_limit=9000, # 2,5 hours
    excluded_model_types=['RecursiveTabular', 'DirectTabular']
)

# Generate predictions on test data
predictions = predictor.predict(test_data)

# Save predictions
predictions.to_csv('autogluon_predictions.csv')

# Leaderboard of models
leaderboard = predictor.leaderboard()
print(leaderboard)